## Part A: Subreddit Prediction ##

Load Subreddit data

In [25]:
subreddit_train = "coursework_subreddit_train.json"
subreddit_test = "coursework_subreddit_test.json"

!curl -o  $subreddit_train https://storage.googleapis.com/tad2018/coursework_subreddit_train3.json
!curl -o  $subreddit_test https://storage.googleapis.com/tad2018/coursework_subreddit_test3.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.0M  100 10.0M    0     0  10.0M      0  0:00:01 --:--:--  0:00:01 11.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2722k  100 2722k    0     0  2722k      0  0:00:01 --:--:--  0:00:01 12.7M


In [0]:
import pandas as pd

train_threads = pd.read_json(path_or_buf=subreddit_train, lines=True)
test_threads = pd.read_json(path_or_buf=subreddit_test, lines=True)
# train_threads



Assign labels in to which data is to be classified

In [0]:
train_labels = train_threads['subreddit']
test_labels = test_threads['subreddit']




**Question 1**
Get title and body from each thread

In [0]:
import json
import pandas as pd


def get_title_body(subreddit_train):
  posts_frame_title = []

  # Construct a dataframe, by opening the JSON file line-by-line
  with open(subreddit_train) as jsonfile:
    for i, line in enumerate(jsonfile):
      thread = json.loads(line)
      posts_train_title=[]

      for post in thread['posts']:

        posts_train_title.append(post.get('body',""))
      posts_train_title.append(thread['title'])
      s =' '.join(posts_train_title)
      posts_frame_title.append(s)

  post_frame= pd.DataFrame(posts_frame_title)
  post_frame.columns=["data"]

  return post_frame

In [0]:
post_frame_train = get_title_body(subreddit_train)
post_frame_test = get_title_body(subreddit_test)

**Count Vectorizer** transform body and title to features

In [30]:

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
one_hot_vectorizer = CountVectorizer()
train_term_document_matrix_cv = one_hot_vectorizer.fit_transform(post_frame_train['data'])
train_term_document_matrix_cv.shape

(1456, 27437)

**Tfidf Vectorizer** transform body and title to features

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_term_document_matrix_tfidfvc = vectorizer.fit_transform(post_frame_train['data'])
train_term_document_matrix_tfidfvc.shape


(1456, 27437)

**Logistic Regression** Learn two models of logistic regression with both the both above vectorizers

In [32]:

from  sklearn.linear_model import LogisticRegression

L1 = LogisticRegression()

model_lr_cv= L1.fit(train_term_document_matrix_cv,train_labels)
test_term_document_matrix_lr_cv= one_hot_vectorizer.transform(post_frame_test['data'])
model_lr_cv.score(test_term_document_matrix_lr_cv, test_labels)



0.673972602739726

In [33]:
L2 = LogisticRegression()
model_lr_tf= L2.fit(train_term_document_matrix_tfidfvc,train_labels)
test_term_document_matrix_lr_tf= vectorizer.transform(post_frame_test['data'])
model_lr_tf.score(test_term_document_matrix_lr_tf, test_labels)


0.4849315068493151

**SVC** Learn two models of logistic regression with both the both above vectorizers

In [34]:
from sklearn.svm import SVC


clf1 = SVC()
clf1.fit(train_term_document_matrix_cv,train_labels,sample_weight=None)
test_term_document_matrix_svc_cv = one_hot_vectorizer.transform(post_frame_test['data'])
clf1.score(test_term_document_matrix_svc_cv, test_labels)


0.2821917808219178

In [35]:
clf2 = SVC()
clf2.fit(train_term_document_matrix_tfidfvc,train_labels,sample_weight=None)
test_term_document_matrix_svc_tf = vectorizer.transform(post_frame_test['data'])
clf2.score(test_term_document_matrix_svc_tf, test_labels)


0.23013698630136986

Evaluate the above models:

In [0]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
%matplotlib inline 
import matplotlib.pyplot as plt

def eval_summary(name, predictions, labels):
  
  
  precision = precision_score(predictions, labels,average='macro')
  recall = recall_score(predictions, labels,average='macro')
  accuracy = accuracy_score(predictions, labels)
  f1 = fbeta_score(predictions, labels, 1,average='macro')#1 means f_1 measure
  
  print("Classifier '%s' has P=%0.3f R=%0.3f Acc=%0.3f F1=%0.3f" % (name,precision,recall,accuracy,f1))


#   print(classification_report(predictions, labels))
#   print('\nConfusion matrix:\n',confusion_matrix(labels, predictions))


Eval scores and confusion  matrix of Logistic regression

In [37]:
eval_summary("CV LR",  model_lr_cv.predict(test_term_document_matrix_lr_cv), test_labels)
eval_summary("TF LR",  model_lr_tf.predict(test_term_document_matrix_lr_tf), test_labels)


Classifier 'CV LR' has P=0.562 R=0.693 Acc=0.674 F1=0.587
Classifier 'TF LR' has P=0.265 R=0.484 Acc=0.485 F1=0.289


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Eval scores and confusion matrix of SVClassifier

In [38]:
eval_summary("CV SVC",  clf1.predict(test_term_document_matrix_svc_cv), test_labels)
eval_summary("TF SVC",  clf2.predict(test_term_document_matrix_svc_tf), test_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Classifier 'CV SVC' has P=0.090 R=0.105 Acc=0.282 F1=0.075
Classifier 'TF SVC' has P=0.050 R=0.012 Acc=0.230 F1=0.019


Evaluation measures of Dummy classifier for both statified and most_frequent strategy

In [39]:
from sklearn.dummy import DummyClassifier

dummy_cv_lr = DummyClassifier(strategy='stratified')
prior_model1 = dummy_cv_lr.fit(train_term_document_matrix_cv,train_labels)
eval_summary("Dummy Random", prior_model1.predict(test_term_document_matrix_lr_cv), test_labels )

dummy_tf_lr = DummyClassifier(strategy='stratified')
prior_model2 = dummy_tf_lr.fit(train_term_document_matrix_tfidfvc,train_labels)
eval_summary("Dummy Random", prior_model2.predict(test_term_document_matrix_lr_tf), test_labels )

dummy_cv_svc = DummyClassifier(strategy='most_frequent')
prior_model3 = dummy_cv_svc.fit(train_term_document_matrix_cv, train_labels)
eval_summary("Dummy MF", prior_model3.predict(test_term_document_matrix_lr_cv), test_labels )



dummy_tf_svc = DummyClassifier(strategy='most_frequent')
prior_model4 = dummy_tf_svc.fit(train_term_document_matrix_tfidfvc,train_labels)
eval_summary("Dummy MF", prior_model4.predict(test_term_document_matrix_lr_tf), test_labels )

Classifier 'Dummy Random' has P=0.039 R=0.041 Acc=0.085 F1=0.039
Classifier 'Dummy Random' has P=0.050 R=0.044 Acc=0.088 F1=0.046
Classifier 'Dummy MF' has P=0.050 R=0.012 Acc=0.230 F1=0.019
Classifier 'Dummy MF' has P=0.050 R=0.012 Acc=0.230 F1=0.019


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


**Question 2** Tuning the parameters of tfidf vectorizer, logistic regressor and SVC

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipeline1=Pipeline([('tfidf',TfidfVectorizer()),('clf',SVC())])


parameters1 = {'tfidf__sublinear_tf':[True,False], 'tfidf__ngram_range':[(1,1),(1,2),(1,3)], 'tfidf__max_features':[5000, 10000, 20000,None],'clf__C':[0.1,1,10,100]}



pipeline2=Pipeline([('tfidf',TfidfVectorizer()),('lr',LogisticRegression())])


parameters2 = {'tfidf__sublinear_tf':[True,False], 'tfidf__ngram_range':[(1,1),(1,2),(1,3)], 'tfidf__max_features':[5000, 10000, 20000,None],'lr__C':[0.1,1,10,100]}


grid1 = GridSearchCV(pipeline1, parameters1)
grid1.fit(post_frame_train['data'],train_labels)    
grid1.predict(post_frame_test['data'])




grid2=GridSearchCV(pipeline2,parameters2)
grid2.fit(post_frame_train['data'],train_labels)
grid2.predict(post_frame_test['data'])



print("Best: %f using %s" % (grid1.best_score_,grid1.best_params_))

print("Best: %f using %s" % (grid2.best_score_,grid2.best_params_))

means = grid1.cv_results_['mean_test_score']
stds = grid1.cv_results_['std_test_score']
params = grid1.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) with: %r" % (mean, stdev, param))

sorted(grid1.cv_results_.keys())

means = grid2.cv_results_['mean_test_score']
stds = grid2.cv_results_['std_test_score']
params = grid2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) with: %r" % (mean, stdev, param))

sorted(grid2.cv_results_.keys())




Best: 0.259615 using {'clf__C': 100, 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 3), 'tfidf__sublinear_tf': False}
Best: 0.722527 using {'lr__C': 100, 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 1), 'tfidf__sublinear_tf': True}
0.229396 (0.002090) with: {'clf__C': 0.1, 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 1), 'tfidf__sublinear_tf': True}
0.229396 (0.002090) with: {'clf__C': 0.1, 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 1), 'tfidf__sublinear_tf': False}
0.229396 (0.002090) with: {'clf__C': 0.1, 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 2), 'tfidf__sublinear_tf': True}
0.229396 (0.002090) with: {'clf__C': 0.1, 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 2), 'tfidf__sublinear_tf': False}
0.229396 (0.002090) with: {'clf__C': 0.1, 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 3), 'tfidf__sublinear_tf': True}
0.229396 (0.002090) with: {'clf__C': 0.1, 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 3

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_lr__C',
 'param_tfidf__max_features',
 'param_tfidf__ngram_range',
 'param_tfidf__sublinear_tf',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

## Part B: Discourse prediction ##

In [1]:
discourse_train = "coursework_discourse_train.json"
discourse_test = "coursework_discourse_test.json"

!curl -o  $discourse_train https://storage.googleapis.com/tad2018/coursework_discourse_train.json
!curl -o  $discourse_test https://storage.googleapis.com/tad2018/coursework_discourse_test.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 59.9M  100 59.9M    0     0  59.9M      0  0:00:01  0:00:01 --:--:-- 32.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.3M  100 15.3M    0     0  7856k      0  0:00:02  0:00:02 --:--:-- 7185k


In [0]:
# The reddit thread structure is nested with posts in a new content.
# This block reads the file as json and creates a new data frame.
import pandas as pd
import json

def load_posts(file):
  # A temporary variable to store the list of post content.
  posts_tmp = list()

  with open(file) as jsonfile:
    for i, line in enumerate(jsonfile):
     # if (i > 2): break
      thread = json.loads(line)
      for post in thread['posts']:
        # NOTE: This could be changed to use additional features from the post or thread.
        # DO NOT change the labels for the test set.
        posts_tmp.append((thread['subreddit'], thread['title'], thread['url'],
                        post['id'], post.get('author', ""), post.get('body', ""), post.get("majority_link", ""), 
                        post.get('post_depth', 0), post.get('author', ""), post.get('majority_type', ""), post.get('in_reply_to', "") ))
       

# Create the posts data frame.  
  labels = ['subreddit', 'title', 'url', 'id', 'author', 'body', 'majority_link', 
          'post_depth', 'author', 'discourse_type', 'in_reply_to']
  return pd.DataFrame(posts_tmp, columns=labels)

In [3]:
train_posts = load_posts(discourse_train)
# Filter out empty labels
train_posts = train_posts[train_posts['discourse_type'] != ""]
print(train_posts.head())
print("Num posts: ", train_posts.size)

  subreddit                                              title  \
0    guitar  Long time Acoustic player here. Recently got a...   
1    guitar  Long time Acoustic player here. Recently got a...   
2    guitar  Long time Acoustic player here. Recently got a...   
3    guitar  Long time Acoustic player here. Recently got a...   
4     apple                                  Loving Continuity   

                                                 url          id  \
0  https://www.reddit.com/r/Guitar/comments/ev3ju...    t3_ev3ju   
1  https://www.reddit.com/r/Guitar/comments/ev3ju...  t1_c1b7ilz   
2  https://www.reddit.com/r/Guitar/comments/ev3ju...  t1_c1b7ioh   
3  https://www.reddit.com/r/Guitar/comments/ev3ju...  t1_c1b7ls3   
4  https://www.reddit.com/r/apple/comments/2jq6on...   t3_2jq6on   

            author                                               body  \
0      SilverChaos  I've been playing around with it for a while. ...   
1  AcrimoniousFuzz  Advice for playing faster: P

The label for the post we will be predicting is in the discourse_type column.

In [4]:
test_posts = load_posts(discourse_test)
# Filter out empty labels
test_posts = test_posts[test_posts['discourse_type'] != ""]
print("Num posts: ", test_posts.size)


Num posts:  220363


In [5]:
train_labels = train_posts['discourse_type']
test_labels = test_posts['discourse_type']
print(len(test_labels))

20033


Examine the distribution over labels on the training data.

In [6]:
discourse_counts = train_labels.value_counts()
print(discourse_counts.describe())

top_discourse = discourse_counts.nlargest(200)
print(top_discourse)
top_discourse = top_discourse.index.tolist()
print(top_discourse)

count       10.000000
mean      7904.600000
std       9654.256841
min       1240.000000
25%       1665.000000
50%       3195.500000
75%      11851.250000
max      31408.000000
Name: discourse_type, dtype: float64
answer              31408
elaboration         14715
question            13497
appreciation         6914
agreement            3804
disagreement         2587
humor                1806
other                1618
announcement         1457
negativereaction     1240
Name: discourse_type, dtype: int64
['answer', 'elaboration', 'question', 'appreciation', 'agreement', 'disagreement', 'humor', 'other', 'announcement', 'negativereaction']


**Question 3** Implementing the logistic regression classifier with the best observed vectorizer(count vectorizer here) representation

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
best_vect = CountVectorizer()

X_train = best_vect.fit_transform(train_posts['body'])


In [0]:

from  sklearn.linear_model import LogisticRegression
L = LogisticRegression()
model_lr_cv= L.fit(X_train,train_labels)

X_test = best_vect.transform(test_posts['body'])

In [49]:
eval_summary("CV LR", model_lr_cv.predict(X_test), test_labels)


Classifier 'CV LR' has P=0.256 R=0.410 Acc=0.482 F1=0.277


**Question 4** Implementing additional features

**Feature 1** **Conent + punctuation** 
Words unigrams, bigrams, trigrams

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1,3))

train_posts_features_contents = vect.fit_transform(train_posts['body'])
test_posts_features_contents = vect.transform(test_posts['body'])


**Feature 2 Structure** No.of sentences,words & characters in title and body

In [0]:
def structurefeatures(train_posts):

 
  words_body=[]
  words_title=[]
  char_body=[]
  sent_body=[]
  sent_title=[]
  char_title=[]
  temp_features=[]
  for row in train_posts.itertuples():
 
    words_body.append(len(row.body.split(' ')))
    sent_body.append(len(row.body.split('.')))
    char_body.append(len(list(row.body)))
    words_title.append(len(row.title.split(' ')))
    sent_title.append(len(row.title.split('.')))
    char_title.append(len(list(row.title)))

  
  temp_features.append((sent_title,words_title,char_title,sent_body,words_body,char_body))
 
  features_data={'structure_sentences_title':sent_title,'structure_words_title':words_title,'structure_char_title':char_title,'structure_sentences_body':sent_body,'structure_words_body':words_body,'structure_char_body':char_body}
  return pd.DataFrame(data=features_data)


In [9]:
train_posts_features = structurefeatures(train_posts)
test_posts_features = structurefeatures(test_posts)
print(train_posts_features.isnull().values.any())
print(test_posts_features.isnull().values.any())


False
False


**Feature 3: ** Author

In [0]:
def author_feature(train_posts):

  author_match = []
  train_posts_features=pd.DataFrame()
 
  line=train_posts.head(1)['author']
  line  = line.T.drop_duplicates().T
 
  for row in train_posts.itertuples():

    if(row.post_depth==0):
      line.author=row.author

    if((line.author==row.author).bool()):
      author_match.append('1')
    else:
      author_match.append('0')
  return pd.DataFrame(author_match)



In [11]:
train_posts_features['author']=author_feature(train_posts)
test_posts_features['author']=author_feature(test_posts)
print(train_posts_features.isnull().values.any())
print(test_posts_features.isnull().values.any())

False
False


**Feature 4** Thread Features

In [0]:
def thread_features(train_posts):
  temp = list()
  temp.append('guitar')
  comment=0
  num_comments = list()
  for row in train_posts.itertuples():


    if(row.post_depth==0):
      
      for i in range (comment):
        num_comments.append(comment-1)
#         print(num_comments)
      comment = 1
      temp=row.subreddit

    if((temp==row.subreddit)):
      comment+=1
    

  return pd.DataFrame(num_comments)

  

In [13]:
train_posts_features['thread_features'] = thread_features(train_posts)
test_posts_features['thread_features'] = thread_features(test_posts)

train_posts_features['thread_features']
print(train_posts_features.isnull().values.any())
print(test_posts_features.isnull().values.any())

False
False


**Feature 5:** Community

In [0]:
from sklearn.preprocessing import LabelEncoder


e = LabelEncoder()

train_community = e.fit_transform(train_posts['subreddit'])
test_community = e.fit_transform(test_posts['subreddit'])

In [0]:
train_posts_features['community']=train_community
test_posts_features['community']=test_community

**Feature 6: **Word2Vec/NLP - Average/ Max of gensim word2vec ​ reddit ​ embeddings for the post 

In [0]:

import nltk

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')



def tokenize(string):
  normalized_tokens = list()
  tokens = tokenizer.tokenize(string)
  for token in tokens:
    normalized = token.lower()
    normalized_tokens.append(normalized)
  return normalized_tokens


all_posts_tokenized_train= train_posts.body.apply(tokenize)
all_posts_tokenized_test= test_posts.body.apply(tokenize)

In [17]:

!pip install 'gensim==3.2.0'
import gensim

model = gensim.models.Word2Vec(all_posts_tokenized_train, size=50, window=5, \
                               min_count=5, sg=0, alpha=0.025, iter=5, batch_words=10000)







    100% |████████████████████████████████| 15.9MB 82kB/s 
    100% |████████████████████████████████| 133kB 5.6MB/s 
    100% |████████████████████████████████| 1.4MB 943kB/s 
    100% |████████████████████████████████| 61kB 9.3MB/s 
    100% |████████████████████████████████| 4.1MB 319kB/s 
    100% |████████████████████████████████| 552kB 2.3MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /content/.cache/pip/wheels/36/48/35/97efc2bd1b233627131c9a936c9de23681846db707b907d353
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/31/9c/20/996d65ca104cbca940b1b053299b68459391c01c774d073126
Successfully built smart-open bz2file


In [0]:
import numpy as np

class Word2VecVectorizer(object):
    def __init__(self, embedding_model):
        self.embedding = embedding_model
        self.dimension = embedding_model.vector_size

    def fit(self, X, y):
        return self
      
    def transform(self, X):  
      # Skip OOV terms. Return 0 if no words are in the vocabulary.
      return np.array([ 
          np.mean([self.embedding.wv[t] for t in tokenize(doc) if t in self.embedding.wv.vocab]
                or [np.zeros(self.dimension)], axis=0)
          for doc in X
      ])

vectorizer = Word2VecVectorizer(model)
train_w2v_post_vector_matrix = vectorizer.transform(train_posts['body'])
test_w2v_post_vector_matrix = vectorizer.transform(test_posts['body'])

Check the shapes of feature matrices and make sure all are of same datatype

In [20]:
print(train_w2v_post_vector_matrix.shape)
train_posts_features = train_posts_features.astype(float)
train_posts_features_contents = train_posts_features_contents.astype(float)
test_posts_features = test_posts_features.astype(float)
test_posts_features_contents = test_posts_features_contents.astype(float)
print(train_posts_features.shape)
print(train_posts_features_contents.shape)



print(test_w2v_post_vector_matrix.shape)
print(test_posts_features.shape)
print(test_posts_features_contents.shape)


(79046, 50)
(79046, 9)
(79046, 3422405)
(20033, 50)
(20033, 9)
(20033, 3422405)


Combine all the features together and train a logistic regression model

In [0]:
from scipy import sparse
from sklearn.linear_model import LogisticRegression

X_train = sparse.hstack((train_w2v_post_vector_matrix,train_posts_features,train_posts_features_contents))
X_test = sparse.hstack((test_w2v_post_vector_matrix,test_posts_features,test_posts_features_contents))


lrcv_w2v = LogisticRegression(max_iter = 300)
lrcv_w2v_model = lrcv_w2v.fit(X_train, train_labels)
  
  

**Eval Summary of model with trained features**

In [25]:
eval_summary("LR TFIDF w/ W2V", lrcv_w2v_model.predict(X_test), test_labels)


Classifier 'LR TFIDF w/ W2V' has P=0.304 R=0.503 Acc=0.563 F1=0.330


**Leave one out model except Word2Vec**

In [65]:
from scipy import sparse
from sklearn.linear_model import LogisticRegression

train_features = sparse.hstack((train_posts_features,train_posts_features_contents))
test_features = sparse.hstack((test_posts_features,test_posts_features_contents))


looclf = LogisticRegression()
looclf.fit(train_features,train_labels)
eval_summary("LOO CLF", looclf.predict(test_features), test_labels)

Classifier 'LOO CLF' has P=0.281 R=0.494 Acc=0.559 F1=0.300


**Leave one out model without Contents**

In [66]:
from scipy import sparse
from sklearn.linear_model import LogisticRegression

train_features = sparse.hstack((train_posts_features,train_w2v_post_vector_matrix))
test_features = sparse.hstack((test_posts_features,test_w2v_post_vector_matrix))


looclf = LogisticRegression()
looclf.fit(train_features,train_labels)
eval_summary("LOO CLF", looclf.predict(test_features), test_labels)

Classifier 'LOO CLF' has P=0.206 R=0.333 Acc=0.499 F1=0.196


**Leave one out model without structure**

In [68]:
from scipy import sparse
from sklearn.linear_model import LogisticRegression

train_features = sparse.hstack((train_posts_features[['author','thread_features','community']],train_w2v_post_vector_matrix,train_posts_features_contents))
test_features = sparse.hstack((test_posts_features[['author','thread_features','community']],test_w2v_post_vector_matrix,test_posts_features_contents))


looclf = LogisticRegression()
looclf.fit(train_features,train_labels)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [69]:
eval_summary("LOO CLF", looclf.predict(test_features), test_labels)

Classifier 'LOO CLF' has P=0.294 R=0.508 Acc=0.561 F1=0.317


**Leave one out model without community**

In [70]:

from scipy import sparse
from sklearn.linear_model import LogisticRegression

train_features = sparse.hstack((train_posts_features[['structure_sentences_title','structure_words_title','structure_char_title','structure_sentences_body','structure_words_body','structure_char_body','author','thread_features']],train_w2v_post_vector_matrix,train_posts_features_contents))
test_features = sparse.hstack((test_posts_features[['structure_sentences_title','structure_words_title','structure_char_title','structure_sentences_body','structure_words_body','structure_char_body','author','thread_features']],test_w2v_post_vector_matrix,test_posts_features_contents))


looclf = LogisticRegression()
looclf.fit(train_features,train_labels)
eval_summary("LOO CLF", looclf.predict(test_features), test_labels)


Classifier 'LOO CLF' has P=0.312 R=0.507 Acc=0.561 F1=0.341


**Leave one out model without thread_features**

In [71]:
from scipy import sparse
from sklearn.linear_model import LogisticRegression

train_features = sparse.hstack((train_posts_features[['structure_sentences_title','structure_words_title','structure_char_title','structure_sentences_body','structure_words_body','structure_char_body','author','community']],train_w2v_post_vector_matrix,train_posts_features_contents))
test_features = sparse.hstack((test_posts_features[['structure_sentences_title','structure_words_title','structure_char_title','structure_sentences_body','structure_words_body','structure_char_body','author','community']],test_w2v_post_vector_matrix,test_posts_features_contents))


looclf = LogisticRegression()
looclf.fit(train_features,train_labels)
eval_summary("LOO CLF", looclf.predict(test_features), test_labels)

Classifier 'LOO CLF' has P=0.299 R=0.486 Acc=0.564 F1=0.325


**Leave one out without author**

In [72]:
from scipy import sparse
from sklearn.linear_model import LogisticRegression

train_features = sparse.hstack((train_posts_features[['structure_sentences_title','structure_words_title','structure_char_title','structure_sentences_body','structure_words_body','structure_char_body','thread_features','community']],train_w2v_post_vector_matrix,train_posts_features_contents))
test_features = sparse.hstack((test_posts_features[['structure_sentences_title','structure_words_title','structure_char_title','structure_sentences_body','structure_words_body','structure_char_body','thread_features','community']],test_w2v_post_vector_matrix,test_posts_features_contents))


looclf = LogisticRegression()
looclf.fit(train_features,train_labels)
eval_summary("LOO CLF", looclf.predict(test_features), test_labels)

Classifier 'LOO CLF' has P=0.283 R=0.461 Acc=0.548 F1=0.304
